In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('Citywide_Mobility_Survey_-_Trip_Survey_2019.csv')
df = pd.read_csv('Citywide_Mobility_Survey_-_Trip_Survey_2019.csv')


In [ ]:
print(df)

In [ ]:
print(df.shape)

In [ ]:
df.head(5)

In [ ]:
#筛选需要的字段
select_cols = ['person_id','person_num','day_num','trip_id','trip_num','trip_duration','trip_distance','o_purpose','o_purpose_category','d_purpose','d_purpose_category','num_non_hh_travelers','num_hh_travelers',"num_travelers","mode_type","mode_1","mode_2","mode_3","mode_4","driver","park_location","park_pay","park_cost","used_transit","transit_access","taxi_cost","taxi_cost_amount","bike_park_location"]
select_df_data = df[select_cols]

In [ ]:
select_cols_1019 = ['person_id','person_num','travel_data_dow','d_purpose','d_purpose_category']
selected_data_1019 = df[select_cols_1019]
selected_data_1019.to_csv('test_data.csv')


In [ ]:
print(select_df_data)

In [ ]:
select_df_data.isnull().any()

In [ ]:
#select_df_data.isnull().sum()
select_df_data[select_df_data.isnull().values==True]

In [ ]:

import tensorflow as tf
import tensorboard
from tensorflow import feature_column
from tensorflow.keras import layers,Sequential
from tensorflow.keras.layers import DenseFeatures,Dense
from sklearn.model_selection import train_test_split
print(tf.__version__)

In [ ]:
#通过sklearn-train_test_split划分数据集测试集验证集
train_data,test_data = train_test_split(select_df_data,test_size=0.2)
train_data,val_data=train_test_split(train_data,test_size=0.2)
print(len(train_data), '训练数据')
print(len(val_data), '验证数据 ')
print(len(test_data), '测试数据')
print(type(train_data))

In [ ]:
#将dataframe存储为csv文件

train_data.to_csv('train_data.csv')
val_data.to_csv('val_data.csv')
test_data.to_csv('test_data.csv')

In [ ]:
print(train_data)

In [ ]:
print(train_data.head())

In [ ]:
#tensorflow能识别的dataset格式
def get_dataset(df,shuffle=True,batch_size=32):
    df=df.copy()
    labels=df.pop('o_purpose')
    ds = tf.data.Dataset.from_tensor_slices((dict(df),labels))
    
    if shuffle:
        ds = ds.shuffle(buffer_size=len(df))
    ds = ds.batch(batch_size)
    return ds

In [ ]:
batch_size = 5
train_ds = get_dataset(train_data,batch_size=batch_size)
val_ds = get_dataset(val_data,shuffle=False,batch_size=batch_size)
test_ds = get_dataset(test_data,shuffle=False,batch_size=batch_size)
print(type(train_ds))

In [ ]:
#分离特征值和标签
ndarray_data = select_df_data.values
feature = ndarray_data[:,1:]
label = ndarray_data[:,0]
print(ndarray_data)
print(feature)
print(label)

In [ ]:
#特征值标准处理化
from sklearn import preprocessing
minmax_scale = preprocessing.MinMaxScaler(feature_range=(0, 1))
norm_features = minmax_scale.fit_transform(feature)

In [ ]:
#定义数据预处理函数
def pre_data(df_data):
    df = df_data.drop(df_data['park_cost'], axis=1)
    df = df_data
    
    ndarray_data = df.values

    features = ndarray_data[:, 1:]
    label = ndarray_data[:, 0]
    
    minmax_scale =preprocessing.MinMaxScaler(feature_range=(0,1))
    norm_features=minmax_scale.fit_transform(features)


In [ ]:
select_df_data[:3]

In [ ]:
pre_data(select_df_data)

In [ ]:
shuffled_df_data = select_df_data.sample(frac=1)

x_data, y_data = pre_data(shuffled_df_data)

train_size = int(len(x_data)*0.8)

x_train = x_data[:train_size]
y_train = y_data[:train_size]

x_test = x_data[train_size:]
y_test = y_data[train_size:]

In [ ]:
#特征值的标签作为输入要素
for feature_batch,label_batch in train_ds.take(1):
    feature = list(feature_batch.keys())
    print('feature list:',feature)
    print('A batch of targets:',label_batch)

In [ ]:
for header in feature:
    print(header)

In [ ]:
#遍历特征列表feature,拿到每一个特征值，
# 使用feature_column.numeric_column(header)将所有类型转为数值类型，
# 在通过DenseFeatures()转为Dense类型，作为输入层
feature_columns = []

for header in feature:
    feature_columns.append(tf.feature_column.numeric_column(header))
feature_layer = DenseFeatures(feature_columns)
#print(feature_layer(next(iter(train_ds))[0]).numpy())
#
#print(type(feature_column.numeric_column(header)))  # 可以看到转换类型
#<class 'tensorflow.python.feature_column.feature_column_v2.NumericColumn'>
#print(type(feature_layer))
#<class 'tensorflow.python.keras.feature_column.dense_features_v2.DenseFeatures'>


In [ ]:
#tensorflow    tf.data.experimental.make_csv_dataset 
def get_dataset(csv_path):
    return tf.data.experimental.make_csv_dataset(
        csv_path,
        batch_size=12,
        label_name='o_purpose',
        na_value='?',
        num_epochs=1,
        ignore_errors=True
    )
    
train_set = get_dataset('train_data.csv'),
test_set = get_dataset('test_data.csv'),
#feats,lbls = next(iter(test_set))
#print('features =', feats)
#print('labels =', lbls)


In [ ]:
#LSTM
from tensorflow import keras
model=keras.models.Sequential()
model.add(keras.layers.Embedding(output_dim=32,
                                 input_dim=28,
                                 input_length=28))

In [ ]:
#LSTM
#全连接层
model.add(keras.layers.Bidirectional(keras.layers.LSTM(units=8)))
model.add(keras.layers.Dense(units=32,activation='relu'))
model.add(keras.layers.Dropout(0.3))


In [ ]:
#LSTM
#输出层
model.add(keras.layers.Dense(units=2,activation='softmax'))
model.summary()


In [ ]:
#模型训练 告知训练时使用的优化器，损失函数，准确率评估标准
#one-hot多分类模型 损失函数用categoricalcrossentropy
#不是多分类的one-hot 编码模型 损失函数用sparse_categorical_crossentropy
#二分类用binary_crossentropy
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.003),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
#模型训练
train_history = model.fit(train_data,
                          validation_split=0.2,
                          epochs=100,
                          batch_size=40,
                          verbose=2
                          )


In [ ]:
import matplotlib as plt

def visu_train_history(train_history,train_metric,validation_metric):
    plt.plot(train_history.history[train_metric])
    plt.plot(train_history.history[validation_metric])
    plt.title('Train History')
    plt.ylabel('train_metric')
    plt.xlabel('epoch')
    plt.legend(['train','validation'],loc='upper left')
    plt.show()

In [ ]:
#建立keras序列模型
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Dense(
                                units=64,
                                input_dim=7,
                                use_bias=True,
                                kernel_initializer='uniform',
                                bias_initializer='zero',
                                activation='relu'
))
model.add(tf.keras.layers.Dense(units=32,
                                activation='sigmoid'))
model.add(tf.keras.layers.Dense(units=1,
                                activation='sigmoid'))

In [ ]:
#模型设置
model.compile(optimizer=tf.keras.optimizers.Adam(0.003),
              loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
#模型训练
train_history = model.fit(x=train_set,
                          y=train_set,
                          validation_split=0.2,
                          epochs=100,
                          batch_size=40,
                          verbose=2)
